In [ ]:
# QX600_Step2.NameMTHdata+AverageConc&Positive.ipynb
# CODE OVERVIEW:
# This code deasl with the Data file after you thresholded and exported from QXManager.
# The input are 2 files: the Thresholded data and the PlateDesign
# This code has 2 main parts. 
    # The first part fill Sample ID based on plate Design. 
    # The second part calculates the average of concerntration/copy number and positive droplets. 
# The output is the data with samplesID and averaged values. 
#TOTAL ACTION NEEDED: 3 

# LAST UPDATED: 26FEB2025 

In [1]:
### FUNCTION: Install and import libraries 
import pandas as pd
import numpy as np

#PART 1 ---------------------------------------------------------------------------------------------------------------------------------------
### FUNCTION: Import files and clean data
################ ACTIONS NEEDED: Change the path to your file location>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Data = pd.read_excel ("/Users/qchen2/Desktop/6X_Plate1_MTH.xlsx")
PlateDesign = pd.read_excel ("/Users/qchen2/Desktop/6X_Plate1_Design.xlsx")
#Need to add a Sample Column and Delete SampleDescriton if data is from QXmanager. 
#If file is from Quantasoft, don't need to run this. 
Data = Data.drop(Data.columns[1:5], axis=1)
Data.insert(1, "Sample","")

### FUNCTION: Fill ID Names based on their location on platedeisgn
# For the location of well, ex: A01, break it into A and 01
# A as 1 on X axis and 01 as 1 on the Y axis. A01 = (1, 1)
# Find (X, Y) on the platedesign and eppend the value of that cell (the matching sampleID) to this well location. 
SampleID = []
for i in range(0,len(Data)):
    WellID = [*Data.iloc[i,0]]
    wid_row = WellID[0]
    x = ord(wid_row) - 65
    y = int(WellID[1]+WellID[2])-1
    PlateDesign.iloc[x,y]
    SampleID.append(PlateDesign.iloc[x,y]) 
Data['Sample'] = SampleID
#If your data is not mataching up for whatever reason, try this code instead. 
#Data['Sample'] = SampleID[1:len(SampleID)]
# If you don't want to average your data and only want the data with ID, run this code
# Data.to_excel('6X_Plate1_MTH_WithID_Averaged.xlsx', index=False)

#PART 2 ---------------------------------------------------------------------------------------------------------------------------------------
### FUNCTION: Calculate the Average Concentraion/copy number and Average positive droplets of the duplicates
MTH = Data
# Add a column in the file to record the average
MTH.insert(4,"Average Conc (copies/uL)",'')
MTH.insert(13,"Average Positives",'')
# Sort the data so that the same sample with the same target/channel/probe color are next to each other
sorted_MTH = MTH.sort_values(by=['Sample','Target'])
# Calculate the averages of Concentration/Copy numbers. 
# Because only numbers has averages, we need to force the file to be read a number instead of a text. 
sorted_MTH['Conc(copies/µL)'] = pd.to_numeric(sorted_MTH['Conc(copies/µL)'], errors='coerce')
sorted_MTH['Average Conc (copies/uL)'] = ""
# Calculate the averages of positive droplets. 
# Because only numbers has averages, we need to force the file to be read a number instead of a text. 
sorted_MTH['Positives'] = pd.to_numeric(sorted_MTH['Positives'], errors='coerce')
sorted_MTH['Average Positives'] = ""
# Delete the averages taken between not the same sample. 
# I acknolwedge that there are better ways to do this. 
mask = (sorted_MTH['Sample'].shift() 
        == sorted_MTH['Sample']) & (sorted_MTH['Target'].shift() == sorted_MTH['Target'])
sorted_MTH.loc[mask, 'Average Conc (copies/uL)'] = (sorted_MTH['Conc(copies/µL)'].shift() + sorted_MTH['Conc(copies/µL)']) / 2
sorted_MTH.loc[mask, 'Average Positives'] = (sorted_MTH['Positives'].shift() + sorted_MTH['Positives']) / 2

################ ACTION NEEDED: Change the name of the file you want to export. >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
sorted_MTH.to_excel('6X_Plate1_MTH_WithID_Averaged.xlsx', index=False)